# Census 2020 response rates

## NewsCounts

In [1]:
import pandas as pd
import numpy as np
import requests
import sys
from datetime import date

In [2]:
today = date.today()

states = pd.read_csv("Data/state_codes.txt",sep=" 	", engine='python')
counties = pd.read_csv("Data/county_codes.txt",sep=" 	", engine='python')

In [3]:
key = "124e1b1d5ceca3b46b3f6bd97d95ea8d4d780e28"

In [4]:
url = f"https://api.census.gov/data/2020/dec/responserate?get=GEO_ID,CAVG&for=state:*&key={key}"
response = requests.get(url)
if response:
    print('State: Success!')
else:
    print('State: An error has occurred.')
    sys.exit("State: "+response.text)

State: Success!


Create data frame from json.

In [5]:
state_df = pd.DataFrame.from_records(response.json()[1:])

state_df.columns = ["GEO_ID","CAVG","state"]
state_df['state'] = state_df.state.astype(int)

state_df['state_name'] = state_df.state.map(states.set_index('code')['state'])

state_df['state_short'] = state_df.state.map(states.set_index('code')['state_code'])

columns = list(state_df.columns)
state_df["date"] = str(today)

state_df = state_df[["date"]+columns]
state_df.sort_values(by="state",inplace=True)
state_df = state_df.reset_index(drop=True)
state_df

,date,GEO_ID,CAVG,state,state_name,state_short
0,2020-03-20,0400000US01,14.7,1,Alabama,AL
1,2020-03-20,0400000US02,2.5,2,Alaska,AK
2,2020-03-20,0400000US04,9.9,4,Arizona,AZ
3,2020-03-20,0400000US05,14,5,Arkansas,AR
4,2020-03-20,0400000US06,12.7,6,California,CA
5,2020-03-20,0400000US08,10.5,8,Colorado,CO
6,2020-03-20,0400000US09,12.9,9,Connecticut,CT
7,2020-03-20,0400000US10,14.9,10,Delaware,DE
8,2020-03-20,0400000US11,14.8,11,NaN,NaN
9,2020-03-20,0400000US12,13.1,12,Florida,FL


In [6]:
state_df.to_csv("Data/sates_response_rates.csv",mode = 'a', header = False, index = False)

I do the same for all counties:

In [7]:
data=[]
url = f"https://api.census.gov/data/2020/dec/responserate?get=GEO_ID,CAVG&for=county:*&key={key}"
response = requests.get(url)
if response:
    print('County: Success!')
    data = response.json()
else:
    print('An error has occurred.')
    sys.exit("County: "+response.text)

County: Success!


In [8]:
county_df = pd.DataFrame.from_records(data[1:])
county_df.columns = data[0]
county_df['state_county'] = county_df.state + county_df.county
county_df['state_county'] = county_df.state_county.astype(int)
county_df['state'] = county_df.state.astype(int)

county_df['state_name'] = county_df.state.map(states.set_index('code')['state'])
county_df['state_short'] = county_df.state.map(states.set_index('code')['state_code'])

county_df['county_name'] = county_df.state_county.map(counties.set_index('code')['county'])

columns = list(county_df.columns)
county_df["date"] = str(today)

county_df = county_df[["date"]+columns]
county_df.sort_values(by=["state_county"],inplace=True)
county_df = county_df.reset_index(drop=True)
county_df

,date,GEO_ID,CAVG,state,county,state_county,state_name,state_short,county_name
0,2020-03-20,0500000US01001,18.4,1,001,1001,Alabama,AL,Autauga
1,2020-03-20,0500000US01003,13.1,1,003,1003,Alabama,AL,Baldwin
2,2020-03-20,0500000US01005,13.6,1,005,1005,Alabama,AL,Barbour
3,2020-03-20,0500000US01007,16.4,1,007,1007,Alabama,AL,Bibb
4,2020-03-20,0500000US01009,16.8,1,009,1009,Alabama,AL,Blount
...,...,...,...,...,...,...,...,...,...
3211,2020-03-20,0500000US72145,0,72,145,72145,Puerto Rico,PR,Vega Baja
3212,2020-03-20,0500000US72147,0,72,147,72147,Puerto Rico,PR,Vieques
3213,2020-03-20,0500000US72149,0,72,149,72149,Puerto Rico,PR,Villalba
3214,2020-03-20,0500000US72151,0,72,151,72151,Puerto Rico,PR,Yabucoa


In [9]:
county_df.to_csv("Data/counties_response_rates.csv",mode = 'a', header = False, index = False)

I do the same for all tracts:

In [10]:
data=[]
c=0
for i in range(len(state_df)):
    s = state_df.iloc[i,:]
    if s.state <10:
        num = f"0{s.state}"
    else:
        num = s.state
    url = f"https://api.census.gov/data/2020/dec/responserate?get=GEO_ID,CAVG&for=tract:*&in=state:{num}&key={key}"
    response = requests.get(url)
    if  not response:
        print('Tract: An error has occurred.')
        print(response.text)
        sys.exit("Tract: "+response.text)
    else:
        start = 0 if c==0 else 1
        data = data + response.json()[start:]
        c=c+1
        
print("Tract: Done!")

Tract: Done!


In [11]:
tract_df = pd.DataFrame.from_records(data[1:])
tract_df.columns = data[0]
tract_df['state_county'] = tract_df.state + tract_df.county
tract_df['state_county'] = tract_df.state_county.astype(int)
tract_df['state'] = tract_df.state.astype(int)

tract_df['state_name'] = tract_df.state.map(states.set_index('code')['state'])
tract_df['state_short'] = tract_df.state.map(states.set_index('code')['state_code'])

tract_df['county_name'] = tract_df.state_county.map(counties.set_index('code')['county'])

columns = list(tract_df.columns)
tract_df["date"] = str(today)

tract_df = tract_df[["date"]+columns]
tract_df.sort_values(by=["state_county","tract"],inplace=True)
tract_df = tract_df.reset_index(drop=True)
tract_df

,date,GEO_ID,CAVG,state,county,tract,state_county,state_name,state_short,county_name
0,2020-03-20,1400000US01001020100,None,1,001,020100,1001,Alabama,AL,Autauga
1,2020-03-20,1400000US01001020200,None,1,001,020200,1001,Alabama,AL,Autauga
2,2020-03-20,1400000US01001020300,None,1,001,020300,1001,Alabama,AL,Autauga
3,2020-03-20,1400000US01001020400,None,1,001,020400,1001,Alabama,AL,Autauga
4,2020-03-20,1400000US01001020501,None,1,001,020501,1001,Alabama,AL,Autauga
...,...,...,...,...,...,...,...,...,...,...
84088,2020-03-20,1400000US72153750501,None,72,153,750501,72153,Puerto Rico,PR,Yauco
84089,2020-03-20,1400000US72153750502,None,72,153,750502,72153,Puerto Rico,PR,Yauco
84090,2020-03-20,1400000US72153750503,None,72,153,750503,72153,Puerto Rico,PR,Yauco
84091,2020-03-20,1400000US72153750601,None,72,153,750601,72153,Puerto Rico,PR,Yauco


In [12]:
tract_df.to_csv("Data/tracts_response_rates.csv",mode = 'a', header = False,  index = False)

In [13]:
response.json()

[['GEO_ID', 'CAVG', 'state', 'county', 'tract'],
 ['1400000US72025201400', None, '72', '025', '201400'],
 ['1400000US72025201000', None, '72', '025', '201000'],
 ['1400000US72025201200', None, '72', '025', '201200'],
 ['1400000US72025201300', None, '72', '025', '201300'],
 ['1400000US72025201500', None, '72', '025', '201500'],
 ['1400000US72025201600', None, '72', '025', '201600'],
 ['1400000US72025201700', None, '72', '025', '201700'],
 ['1400000US72025201800', None, '72', '025', '201800'],
 ['1400000US72025201900', None, '72', '025', '201900'],
 ['1400000US72025202000', None, '72', '025', '202000'],
 ['1400000US72025202100', None, '72', '025', '202100'],
 ['1400000US72025202300', None, '72', '025', '202300'],
 ['1400000US72025202403', None, '72', '025', '202403'],
 ['1400000US72025202404', None, '72', '025', '202404'],
 ['1400000US72025202601', None, '72', '025', '202601'],
 ['1400000US72025202701', None, '72', '025', '202701'],
 ['1400000US72025202702', None, '72', '025', '202702'],